### Introduction
In this code, we calculated the intersection percentage between PRIO GRID cell which has a SNL mining of certain primary commidty and PRIO GRID cell code which has indigenous land on it.

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

### Read indigenous land data

In [2]:
ind_land=pd.read_csv("../input/ind_land.csv",usecols=["GRID IDs"])
print(ind_land.shape)
ind_land= ind_land[pd.to_numeric(ind_land['GRID IDs'], errors='coerce').notnull()]
ind_land["GRID IDs"] = ind_land["GRID IDs"].astype("int")
ind_land.shape

(159050, 1)


/var/folders/0p/mkzn2l513fl51bl6rs96d9z40000gn/T/ipykernel_18711/2092993344.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  ind_land=pd.read_csv("../input/ind_land.csv",usecols=["GRID IDs"])


(156873, 1)

### Find the Prio Grid cell of each indigenous land

Store the PRIO GRID cell id of each indigenous land in ls_1

In [3]:
# find the PRIO GRID cell of each indigenous land and store in ls_1
tmp=[]
def transfer_to_ls(x):
    try:
        tmp.append(int(x))
    except ValueError:
        pass
ind_land['GRID IDs'].apply(lambda x: transfer_to_ls((x)))
ls_1= tmp

### Processing SNL

In [5]:
# drop the invalid data such as location is out of earth boundary 
df_SNL= pd.read_csv("../input/SNL-PRIO GRID IDs.csv")
print(df_SNL.shape)
df_SNL.dropna(subset=['LATITUDE', 'LONGITUDE'], inplace=True)
df_SNL = df_SNL[pd.to_numeric(df_SNL['PRIO GRID ID'], errors='coerce').notnull()]
df_SNL["PRIO GRID ID"] = df_SNL["PRIO GRID ID"].astype("int")
df_WHC = df_SNL[(df_SNL['LATITUDE']<=90) & (df_SNL['LATITUDE']>= -90) &
                (df_SNL['LONGITUDE']<=180) & (df_SNL['LONGITUDE']>= -180)  ].reset_index().drop(columns="index",axis=1)
print(df_SNL.shape)
df_SNL.head()

(39612, 46)
(39370, 46)


/var/folders/0p/mkzn2l513fl51bl6rs96d9z40000gn/T/ipykernel_18711/1284725798.py:2: DtypeWarning: Columns (2,22,23,24,25,26,27,28,30,31,32,33,36,37,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_SNL= pd.read_csv("../input/SNL-PRIO GRID IDs.csv")


Unnamed: 0           Property name Property ID Also Known As  \
0          0           11 RAU-Novaya       68289           NaN   
1          1           12 Mile Creek       66178           NaN   
2          2               14 Karats       65525           NaN   
3          3  14 Mile Well Extension       63665           NaN   
4          4     15 Mile Silver City       65760           NaN   

                                     OWNER(S) Equity ownership %  \
0  Sodeistvie Private Enterprise (Owner) 100%                100   
1               U&D Coal Limited (Owner) 100%                100   
2          Dios Exploration Inc. (Owner) 100%                100   
3               Private Interest (Owner) 100%                100   
4       North Lily Mining Company (Owner) 50%                 50   

  DEVELOPMENT STAGE ACTIVITY STATUS   COMMODITY(S) PRIMARY COMMODITY  ...  \
0        Grassroots        Inactive           Coal              Coal  ...   
1        Grassroots        Inactive           Coal              Coal  ...   
2        Grassroots        Inactive  Gold,Diamonds              Gold  ...   
3        Grassroots        Inactive           Gold              Gold  ...   
4        Grassroots        Inactive    Gold,Silver              Gold  ...   

  Unnamed: 36 Unnamed: 37 Unnamed: 38 Unnamed: 39 Unnamed: 40  Unnamed: 41  \
0         NaN         NaN         NaN         NaN         NaN          NaN   
1         NaN         NaN         NaN         NaN         NaN          NaN   
2         NaN         NaN         NaN         NaN         NaN          NaN   
3         NaN         NaN         NaN         NaN         NaN          NaN   
4         NaN         NaN         NaN         NaN         NaN          NaN   

   Unnamed: 42 Unnamed: 43 Unnamed: 44 PRIO GRID ID  
0          NaN         NaN         NaN       199878  
1          NaN         NaN         NaN        96419  
2          NaN         NaN         NaN       205418  
3          NaN         NaN         NaN        88445  
4          NaN         NaN         NaN       221841  

[5 rows x 46 columns]

In [6]:
subset_df=df_SNL[['PRIO GRID ID', 'PRIMARY COMMODITY']] 
subset_df

PRIO GRID ID PRIMARY COMMODITY
0            199878              Coal
1             96419              Coal
2            205418              Gold
3             88445              Gold
4            221841              Gold
...             ...               ...
39607        106057              Gold
39608        173402            Nickel
39609        189948            Nickel
39610        169787              Zinc
39611        101940       Ferrochrome

[39370 rows x 2 columns]

Get the list of primary commodity in SNL dataset

In [7]:
primary_commodity = pd.unique(subset_df["PRIMARY COMMODITY"]).tolist()
primary_commodity

['Coal',
 'Gold',
 'U3O8',
 'Graphite',
 'Copper',
 'Lithium',
 'Tungsten',
 'Potash',
 'Molybdenum',
 'Lanthanides',
 'Diamonds',
 'Iron Ore',
 'Phosphate',
 'Nickel',
 'Silver',
 'Zinc',
 'Ilmenite',
 'Antimony',
 'Bauxite',
 'Manganese',
 'Lead',
 'Platinum',
 'Tin',
 'Niobium',
 'Chromite',
 'Tantalum',
 'Vanadium',
 'Rutile',
 'Palladium',
 'Cobalt',
 'Heavy Mineral Sands',
 'Titanium',
 'Zircon',
 'Scandium',
 'Alumina',
 'Ferrochrome',
 'Yttrium',
 'Chromium',
 'Aluminum',
 'Ferronickel',
 'Ferrotungsten',
 'Ferromanganese',
 'Ferrovanadium',
 'Platinum Group Metals']

In [8]:
subset_df["PRIO GRID ID"]=subset_df["PRIO GRID ID"].astype("int")

/var/folders/0p/mkzn2l513fl51bl6rs96d9z40000gn/T/ipykernel_18711/534760579.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["PRIO GRID ID"]=subset_df["PRIO GRID ID"].astype("int")


### Find the overlapping between PRIO GRID cell of certain primary commodity SNL mining and indigenous Land 
```
count = 0 
Loop thourhg the PRIO GRID IDs of indigenous lands in ls_1:
    Loop through PRIO GRID IDs of SNL data mining in each primary commodity in ls_2:
        if there is PRIO GRID IDs where has indigenous land and SNL data mining:
            count +=1 
percentage of overlapping between PRIO GRID cell of certain primary commodity SNL mining and indigenous Land = count / SNL total mining count 
```

In [9]:
for i in primary_commodity:
    if i != np.nan:
        df=subset_df.loc[subset_df["PRIMARY COMMODITY"]==i]
        ls_2=[] # find the PRIO GRID cell of each SNL 
                # of certain primary commodity and store in ls_2

        def transfer_to_ls(x):
            try:
                ls_2.append(int(x))
            except ValueError:
                pass
        df['PRIO GRID ID'].apply(lambda x: transfer_to_ls((x)))
        same_ls= []
        for j in ls_1:
            if j in set(ls_2):
                same_ls.append(j)

        print("The overlapping percentage between primary commodity", i, 
                "and Indigenous Lands: ", "%.4f" % ((len((same_ls))/df.shape[0])))
        

The overlapping percentage between primary commodity Coal and Indigenous Lands:  0.5376
The overlapping percentage between primary commodity Gold and Indigenous Lands:  2.3339
The overlapping percentage between primary commodity U3O8 and Indigenous Lands:  0.4367
The overlapping percentage between primary commodity Graphite and Indigenous Lands:  0.8354
The overlapping percentage between primary commodity Copper and Indigenous Lands:  10.4041
The overlapping percentage between primary commodity Lithium and Indigenous Lands:  1.8545
The overlapping percentage between primary commodity Tungsten and Indigenous Lands:  0.5782
The overlapping percentage between primary commodity Potash and Indigenous Lands:  0.0046
The overlapping percentage between primary commodity Molybdenum and Indigenous Lands:  2.3165
The overlapping percentage between primary commodity Lanthanides and Indigenous Lands:  0.4322
The overlapping percentage between primary commodity Diamonds and Indigenous Lands:  0.7353